# Get explainability metrics

This notebook is used to get explainability metrics for the model based on the previously trained model and the computed attributions. The configuration used is the same as the [starter](starter.ipynb) and [explain](explain.ipynb) notebooks.

## Load model and data

In [ ]:
import sys

sys.path.append("../")

import torch
import numpy as np

from beexai.dataset.dataset import Dataset
from beexai.dataset.load_data import load_data
from beexai.evaluate.metrics.get_results import get_all_metrics
from beexai.explanation.explaining import CaptumExplainer
from beexai.training.train import Trainer
from beexai.utils.time_seed import set_seed
from beexai.utils.sampling import stratified_sampling

set_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

DATA_NAME = "kickstarter"
MODEL_NAME = "NeuralNetwork"
CONFIG_PATH = f"config/{DATA_NAME}.yml"
data_test, target_col, task, _ = load_data(from_cleaned=True, config_path=CONFIG_PATH)
scale_params = {
    "x_num_scaler_name": "quantile_normal",
    "x_cat_encoder_name": "ordinalencoder",
    "y_scaler_name": "labelencoder",  # change to minmax or another float scaler for regression
    "cat_not_to_onehot": ["name"],
}
data = Dataset(data_test, target_col)
X_train, X_test, y_train, y_test = data.get_train_test(
    test_size=0.2, scaler_params=scale_params
)
NUM_LABELS = data.get_classes_num(task)

NN_PARAMS = {"input_dim": X_train.shape[1], "output_dim": NUM_LABELS}
trainer = Trainer(MODEL_NAME, task, NN_PARAMS, device)
trainer.load_model(f"../output/models/{DATA_NAME}/{MODEL_NAME}.pt")

TEST_SIZE = 100
X_test, y_test = stratified_sampling(X_test, y_test, TEST_SIZE, task)

## Train Dummy

Train a dummy model on shuffled labels to compare with the real model explanations

In [ ]:
DUMMY_TRAIN_SIZE = 100
X_train_sampled, y_train_sampled = stratified_sampling(
    X_train, y_train, DUMMY_TRAIN_SIZE, task
)
rand_trainer = Trainer(MODEL_NAME, task, NN_PARAMS, device)
X_perm, y_perm = X_train_sampled.values, np.random.permutation(y_train_sampled.values)
rand_trainer.train(X_perm, y_perm)
rand_trainer.model.eval()

## Get metrics

`get_all_metrics` method automatically computes XAI metrics for each instance. Some metrics can be customized by specifying hyperparameters, those used in our paper are set by default.

We also provide the possibility of comparison with random attributions and attributions obtained with the dummy model.


In [ ]:
METHOD = "IntegratedGradients"
exp = CaptumExplainer(
    trainer.model, task=task, method=METHOD, sklearn=False, device=device
)
exp.init_explainer()
attributions = exp.compute_attributions(
    X_test, DATA_NAME, MODEL_NAME, METHOD, "../output/explain/", use_abs=False
)

rand_exp = CaptumExplainer(
    rand_trainer.model, task=task, method=METHOD, sklearn=False, device=device
)
rand_exp.init_explainer()
rand_attributions = rand_exp.compute_attributions(
    X_test, DATA_NAME, MODEL_NAME, METHOD, "../output/explain/", use_abs=False
)

all_preds = trainer.model.predict(X_test.values)

get_all_metrics(
    X_test,
    all_preds,
    trainer.model,
    exp,
    rand_trainer.model,
    rand_exp,
    print_plot=True,
    auc_metric="accuracy",
    device=device,
    use_random=True,
    use_ref=True,
    attributions=attributions,
    attributions_ref=rand_attributions,
)